In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb
os.chdir(main)
%run stdPlotting.ipynb
os.chdir(d['py'])
from mCGE import NCP_CGE as CGE

# Comparing Emission Regulation Types

*Load model:*

In [2]:
vName, t0 = 'vLarge', 2019
name = '_'.join([vName, str(t0), 'CGE','EmRegEconWideCapital'])
M = CGE.load(os.path.join(d['data'], name))

*Store baseline solution (with name based on regulation type):*

In [3]:
M.db.mergeInternal()
baseRegulation = M.regulation
dbs = {}
dbs[M.regulation] = M.solve(state = 'B')

## 2. Regulation shocks

### 2.1. Remove emission targets, check for effect of uniform increase in taxes

In [4]:
M.regulation = None
M.m['M'].regulation = None
M.db.aom(adj.rc_pd(M.db('tauCO2agg'), M.db('t0')) * 1.5)

In [5]:
sol_CO2t0 = M.solve(state = 'B')

GamsExceptionExecution: GAMS return code not 0 (3), check C:\Users\sxj477\Documents\GitHub\CGE_Generator\work_folder\_gams_py_gjo1.lst for more details

### 2.2. Optimization with Single-year targets

Adjust type of regulation and re-solve:

In [4]:
M.regulation = 'SYT_OPT'
M.m['M'].regulation = 'SYT_OPT'
dbs[M.regulation] = M.solve(state = 'B')

In [6]:
db = dbs['SYT_OPT']

In [7]:
db('p').xs('L',level='n')

t
2019    0.987591
2020    0.959880
2021    1.018337
2022    0.963115
2023    0.999153
          ...   
2090    0.992688
2091    0.992687
2092    0.992686
2093    0.992684
2094    1.000000
Name: p, Length: 76, dtype: float64

In [8]:
db('p')

t     n      
2019  agg24      1.008454
      agg4       0.997055
      agg3       0.999659
      agg12      1.008038
      agg2       0.995935
                   ...   
2094  agg13_F    1.000000
      agg5_F     1.000000
      agg10_F    1.000000
      K          0.000000
      L          1.000000
Name: p, Length: 4180, dtype: float64

### 2.2. Linear reduction path

Start from the single-year target approach with Hotelling Rule:

In [12]:
M.regulation = 'LRP'
M.m['M'].regulation = 'LRP'
dbs[M.regulation] = M.solve(state = 'B')

### 2.3. Emission budget

Start from the single-year target approach with Hotelling Rule:

In [13]:
M.regulation = 'EB_HR'
M.m['M'].regulation = 'EB_HR'
dbs[M.regulation] = M.solve(state = 'B')

### 2.4. Emission budget, optimal

Adjust type of regulation and re-solve:

In [14]:
M.regulation = 'EB_OPT'
M.m['M'].regulation = 'EB_OPT'
dbs[M.regulation] = M.solve(state = 'B')

Compute abatement capital from this formulation: